In [1]:
import time
import requests
import json

In [2]:
import utils
#from importlib import reload

### Connection elastic search

In [3]:
print("Connection à elasticsearch")

Connection à elasticsearch


In [4]:
# crée une connection elasticsearch
es = utils.connect_elasticsearch()

--> elasticsearch bien connecté


In [5]:
# infos connection
#dir(es)
#es.info()

### Initialisation

In [24]:
# variables grlobales

# nom de l'index
index_name = "python_traffic"

# ré-çrée l'index ou maj
index_init = False

# nombre de ligne par requête
traffic_nb_rows = 100

# niveau de confiance des données
traffic_reliability = 5

# intervalle de requêtage et temps maximale
traffic_time_interval = 10
traffic_time_max = 30

### Initialisation de l'index et mapping
Méthodologie à adopter pour `index_init` :
* True : si on crée un nouvel index
* False : si on met à jour directement l'index existant

In [7]:
print("Gestion de l'index", index_name)

Gestion de l'index python_traffic


In [8]:
index_create = False
if not index_init:
    if es.indices.exists(index=index_name):
        index_init_text = "mise à jour"
    else:
        index_create = True
        index_init_text = "création"
else:
    index_create = True
    if es.indices.exists(index=index_name):
        index_init_text = "remplacement (après suppression)"
        es.indices.delete(index=index_name)
    else:
        index_init_text = "création"

print("--> opéation à réaliser sur l'index:", index_init_text)

--> opéation à réaliser sur l'index: mise à jour


In [9]:
# mappings pour les coordonnées geospatiales
req_body = {
    "mappings": {
        "properties": {
            "fields": {
                "properties": {
                    # fields.geo_point_2d
                    "geo_point_2d": { "type":"geo_point" },
                    
                    # fields.geo_shape
                    "geo_shape": { "type":"geo_shape" }
                }
            },
            
            "geometry": {
                "properties": {
                    # geometry.coordinates
                    "coordinates": { "type":"geo_point" }
                 }
             }
        }
    }
}

In [10]:
# crée l'índex vide
if index_create:
    es.indices.create(index=index_name, body=req_body)
    print("--> index bien créé!")

### Import depuis l'api traffic

In [11]:
print("API traffic")

API traffic


In [12]:
# url
traffic_url = "https://data.rennesmetropole.fr/api/records/1.0/search/?dataset=etat-du-trafic-en-temps-reel&q=&rows="+str(traffic_nb_rows)
print("--> url:", traffic_url)

--> url: https://data.rennesmetropole.fr/api/records/1.0/search/?dataset=etat-du-trafic-en-temps-reel&q=&rows=100


In [13]:
# get
traffic = requests.get(traffic_url)
if traffic.status_code==200:
    print("--> requête ok!")
else:
    print("--> requête non-ok!")
    print("----> erreur :", traffic.status_code)

--> requête ok!


In [14]:
# les données
data_json = traffic.json()
data = data_json["records"]
nb_rows1 = len(data)
print(f"--> nombre de documents importés: {nb_rows1}/{traffic_nb_rows}")

--> nombre de documents importés: 100/100


### Data processing

In [15]:
print("Qualité des données")

Qualité des données


In [16]:
print("--> reliability...")
data = [i for i in data if i["fields"]["traveltimereliability"]>=traffic_reliability]

--> reliability...


In [17]:
nb_rows2 = len(data)
nb_rows3 = nb_rows1-nb_rows2
print(f"----> nombre de documents restants: {nb_rows2}/{nb_rows1} ({nb_rows3} supprimés)")

----> nombre de documents restants: 85/100 (15 supprimés)


### Export vers elasticsearch

In [18]:
print("Export vers elasticsearch")

Export vers elasticsearch


In [19]:
# export vers elastic
print("...")
i = es.count(index=index_name)['count']+1
j = 0
for doc_body in data:
    req = es.index(index=index_name, id=i, doc_type='_doc', body=doc_body)
    i = i+1
    j = j+1
print(f"----> nombre de documents exportés: {j}/{nb_rows2}")

...
----> nombre de documents exportés: 85/85


In [20]:
time.sleep(3)
nb_rows4 = es.count(index=index_name)["count"]
print("--> nombre total de documents dans l'index:", nb_rows4)

--> nombre total de documents dans l'index: 103


In [21]:
print("FIN")

FIN


In [22]:
#es.get(index=index_name, doc_type='_doc', id=4)

Note :
les étapes 'import', 'data processing' et 'export' sont à placer dans un batch et à refaire en boucle.

----